In [1]:
import sys
sys.path.insert(0, ".")


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

from prediksicovidjatim import database
database.init()

In [3]:
from core import mapping
from prediksicovidjatim.mapping import updater
from prediksicovidjatim import database, util
from prediksicovidjatim.data.map import MapDataRepo, MapDataPred, MapDataReal

portal = os.getenv("ARCGIS_PORTAL")
user = os.getenv("ARCGIS_USER")
pw = os.getenv("ARCGIS_PASS")
real_layer_id = os.getenv("REAL_LAYER_ID")

FIRST_TANGGAL_STR = os.getenv("FIRST_TANGGAL") or '2020-03-20'
FIRST_TANGGAL = util.parse_date(FIRST_TANGGAL_STR)

In [4]:
kabko = "KAB. MALANG"
selected_kabko = kabko

In [6]:
mapping.update_map(kabko, update_real=True, update_prediction=True)

Updated maps of KAB. MALANG


In [5]:
updater.GEOMETRY_CACHE[kabko]

KeyError: 'KAB. MALANG'

In [40]:
u = updater.MapUpdater(portal, user ,pw)

In [41]:
real_layer = u.get_layer(real_layer_id)

In [42]:
geometry = u.get_kabko_geometry(real_layer, kabko)
geometry

{'rings': [[[12559793.625, -935436.09375],
   [12559785.96875, -935426.34375],
   [12559784.25, -935422.25],
   [12559783.40625, -935414.53125],
   [12559792.75, -935408.75],
   [12559819.9375, -935408.21875],
   [12559840.3125, -935411.84375],
   [12559850.5, -935418.28125],
   [12559834.375, -935431.71875],
   [12559793.625, -935436.09375]],
  [[12559658.5625, -935484.9375],
   [12559641.59375, -935474.1875],
   [12559630.53125, -935460.375],
   [12559630.53125, -935451.5625],
   [12559654.3125, -935429.46875],
   [12559661.125, -935424.625],
   [12559687.4375, -935416.25],
   [12559718.03125, -935413.90625],
   [12559728.21875, -935418.625],
   [12559733.3125, -935425.8125],
   [12559690, -935472.59375],
   [12559673.84375, -935483.4375],
   [12559658.5625, -935484.9375]],
  [[12560919.78125, -935704.15625],
   [12560909.59375, -935701.15625],
   [12560903.65625, -935687.625],
   [12560921.46875, -935647.28125],
   [12560925.71875, -935638.5],
   [12560937.625, -935616.59375],
   [1

In [43]:
features = u.fetch_kabko_feature_tanggal(real_layer, kabko, geometry)
features[5]

{"geometry": {"rings": [[[12559793.625, -935436.09375], [12559785.96875, -935426.34375], [12559784.25, -935422.25], [12559783.40625, -935414.53125], [12559792.75, -935408.75], [12559819.9375, -935408.21875], [12559840.3125, -935411.84375], [12559850.5, -935418.28125], [12559834.375, -935431.71875], [12559793.625, -935436.09375]], [[12559658.5625, -935484.9375], [12559641.59375, -935474.1875], [12559630.53125, -935460.375], [12559630.53125, -935451.5625], [12559654.3125, -935429.46875], [12559661.125, -935424.625], [12559687.4375, -935416.25], [12559718.03125, -935413.90625], [12559728.21875, -935418.625], [12559733.3125, -935425.8125], [12559690, -935472.59375], [12559673.84375, -935483.4375], [12559658.5625, -935484.9375]], [[12560919.78125, -935704.15625], [12560909.59375, -935701.15625], [12560903.65625, -935687.625], [12560921.46875, -935647.28125], [12560925.71875, -935638.5], [12560937.625, -935616.59375], [12560941.875, -935602], [12560952.90625, -935586.5625], [12560966.5, -935

In [44]:
with database.get_conn() as conn, conn.cursor() as cur:
    real_data = MapDataRepo.fetch_real_data(selected_kabko, cur)
    kabko = MapDataRepo.get_kabko_full(selected_kabko, cur)

real_data = MapDataReal.shift(real_data, FIRST_TANGGAL)
len(real_data)

133

In [30]:
to_update, update_keys = u.to_update(features, real_data)
to_update[3]

{"geometry": {"rings": [[[12611945.152673, -885343.450191033], [12611965.313142, -885256.201476952], [12611960.2713979, -885205.371850239], [12611966.1998346, -885216.712982727], [12611985.1240196, -885240.31557008], [12611945.152673, -885343.450191033]], [[12611945.152673, -885343.450191033], [12612092.6685122, -885295.408805905], [12612147.8721109, -885291.497997373], [12612282.9134853, -885315.130643076], [12612356.7997987, -885322.634070754], [12612457.0162878, -885323.867829502], [12612535.152759, -885316.255641656], [12612593.7532604, -885311.059422039], [12612677.8382909, -885299.321939566], [12612711.8073181, -885286.296322337], [12612853.6422978, -885245.675628997], [12612987.8303169, -885220.431433824], [12613046.5586614, -885216.104393163], [12613119.4719855, -885210.732208336], [12613217.1419238, -885211.373547039], [12613217.3521592, -885211.359598305], [12613325.0026705, -885204.193485279], [12613420.9742075, -885201.834430669], [12613533.9302581, -885212.017513776], [126

In [45]:
util.ms_to_date(to_update[3].attributes["tanggal"])

datetime.date(2020, 3, 23)

In [47]:
u.save(real_layer, selected_kabko, real_data, True)

(133, 100)